In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import ROOT
from ROOT import RooFit
import uproot
from sweights import SWeight
from sweights import convert_rf_pdf

# Import Data

In [ ]:
# In this notebook we only process the main signal and the generic events,
# for illustration purposes.
# You can add other backgrounds after if you wish.
samples = ["Signal", "ccbar"]

In [ ]:
DataFrames = {}  # define empty dictionary to hold dataframes
Date = "1112"
Attempt = "0"

# Signal:
DataFrames[samples[0]] =  uproot.concatenate("/home/belle2/amubarak/C01-Simulated_Events/Ds2D0enu-Signal.root:Dstree",library='pd')
# Background
for s in samples[1:]: # loop over samples
    DataFrames[s] =  uproot.concatenate("/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_" + Date +"24_"+ Attempt +"_"+ s +".root:Dstree",library='pd')

### Setup
The code below will be used to apply cuts to the data.

In [ ]:
# Electron ID
#-------------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['e_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['e_electronID']>=0.95]
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_em_electronID']>=0.95]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_em_electronID']>=0.95]

# Photon Conversion
#-------------------
DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_gammaveto_M_Correction']>=0.1]
DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_gammaveto_M_Correction']>=0.1]

# Peaking Background Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_diff_D0pi']>=0.15)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_diff_D0pi']>=0.15)]

# # Vertex Fitting
# #----------------
# DataFrames["Signal"] = DataFrames["Signal"][DataFrames["Signal"]['Ds_chiProb']>=0.01]
# DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames["ccbar"]['Ds_chiProb']>=0.01]

# Dalitz Removal
#----------------------------
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["ccbar"]['Ds_pi0veto_M_Correction']>=0.16)]
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['Ds_pi0veto_M_Correction']<=0.08) | (DataFrames["Signal"]['Ds_pi0veto_M_Correction']>=0.16)]

# Vertex Fit
#----------------
DataFrames["Signal"] = DataFrames["Signal"][DataFrames[samples[0]]['Ds_chiProb_rank']==1]
DataFrames["ccbar"] = DataFrames["ccbar"][DataFrames[samples[1]]['Ds_chiProb_rank']==1]

# # D0 Invariant Mass
# #-----------------------
# DataFrames["Signal"] = DataFrames["Signal"][(DataFrames["Signal"]['D0_dM']>=-0.02) & (DataFrames["ccbar"]['D0_dM']<=0.02)]
# DataFrames["ccbar"] = DataFrames["ccbar"][(DataFrames["ccbar"]['D0_dM']>=-0.02) & (DataFrames["ccbar"]['D0_dM']<=0.02)]

### Setup the Frame for the Plot

In [ ]:
c = ROOT.TCanvas("fit", "fit", 650, 700)
pad1 = ROOT.TPad("pad1","pad1",0,0.25,1,1)
pad2 = ROOT.TPad("pad2","pad2",0,0,1,0.25)
pad1.SetTopMargin(0.05)
pad1.SetBottomMargin(0.05)
pad1.SetLeftMargin(0.15)
pad1.SetRightMargin(0.05)
pad2.SetLeftMargin(0.15)
pad2.SetRightMargin(0.05)
pad2.SetTopMargin(0.01)
pad2.SetBottomMargin(0.4)
# pad2.SetBorderMode(0);
pad1.Draw()
pad2.Draw()
pad1.cd()
ROOT.gStyle.SetOptStat(0)

### Initialize

In [ ]:
bin = 50
xL = -0.05
xR = 0.05

# Data

In [ ]:
f = ROOT.TFile('/home/belle2/amubarak/C03-Grid/Completed/Ds2D0e-Generic_Ds_111224_0_ccbar.root')
tree = f.Get('Dstree')

dM = ROOT.RooRealVar("D0_dM", "Mass Difference", xL, xR)
Veto = ROOT.RooRealVar("Ds_gammaveto_M_Correction","Veto",0.1,1000)
BCS = ROOT.RooRealVar("Ds_chiProb_rank","BCS",-1,1.25)

data = ROOT.RooDataSet("data", "dataset with mass", ROOT.RooArgSet(dM,Veto,BCS), RooFit.Import(tree))
print(data.Print())

# Build Model

In [ ]:
# Signal PDF:
#---------------------
sig_mean = ROOT.RooRealVar("sig_mean", "mean of crystal", 4.8432e-05, -0.005, 0.005)
sig_sigmaL = ROOT.RooRealVar("sig_sigmaL", "sig_sigmaL", 4.2561e-03, 1e-4, 0.02)
sig_sigmaR =  ROOT.RooRealVar("sig_sigmaR", "sig_sigmaR", 4.2065e-03, 1e-4, 0.02)
sig_alphaL = ROOT.RooRealVar("sig_alphaL", "sig_alphaL", 1.2505e+00, 1e-4, 4.)
sig_alphaR = ROOT.RooRealVar("sig_alphaR", "sig_alphaR", 1.2811e+00, 1e-4, 4.)
sig_nL = ROOT.RooRealVar("sig_nL", "sig_nL", 3.3075e+00, 1e-4, 5)
sig_nR = ROOT.RooRealVar("sig_nR", "sig_nR", 4.9289e+00, 1e-4, 5)
# sig_mean.setConstant()
# sig_sigmaL.setConstant()
# sig_sigmaR.setConstant()
# sig_alphaL.setConstant()
# sig_alphaR.setConstant()
# sig_nL.setConstant()
# sig_nR.setConstant()
Signal = ROOT.RooCrystalBall("Signal", "Signal", dM, sig_mean,sig_sigmaL,sig_sigmaR,sig_alphaL,sig_nL,sig_alphaR,sig_nR)

# Background PDF:
#---------------------
# Constant Background:
c0 = ROOT.RooRealVar("c0", "coefficient #0", 1)
Background = ROOT.RooPolynomial("Background", "background p.d.f.", dM, c0)

# C o n s t r u c t   a   S i g na l   a n d   B a c k g r o u n d   P D F:
# --------------------------------------------------------------
nsig = ROOT.RooRealVar("nsig", "#Signal events", 2.8464e+05, 0., 600000)
nbkg = ROOT.RooRealVar("nbkg", "#background events", 3.0406e+04, 0., 600000)
model = ROOT.RooAddPdf("model", "g+a", ROOT.RooArgSet(Signal, Background), ROOT.RooArgSet(nsig, nbkg))

# Fit

In [ ]:
# Perform fit and save result
r = model.fitTo(data, RooFit.Save(True), RooFit.PrintLevel(-1))

# Plot Model

In [ ]:
# Create a RooPlot to draw on. We don't manage the memory of the returned
# pointer. Instead we let it leak such that the plot still exists at the end of
# the macro and we can take a look at it.
framedM = dM.frame()
data.plotOn( framedM, RooFit.Binning(bin))
model.plotOn(framedM, RooFit.Components("Background"), RooFit.LineStyle(ROOT.kDashed), RooFit.LineColor(46), RooFit.Name("Background"))
model.plotOn(framedM, RooFit.Components("Signal"), RooFit.LineStyle(ROOT.kDashed), RooFit.LineColor(28), RooFit.Name("Signal"))
model.plotOn(framedM, RooFit.LineColor(28), RooFit.Name("model"))

# Calculate chi^2

In [ ]:
# Show the chi^2 of the curve w.r.t. the histogram
# If multiple curves or datasets live in the frame you can specify
# the name of the relevant curve and/or dataset in chiSquare()
npar = model.getParameters(data).selectByAttrib("Constant",ROOT.kFALSE).getSize() #select floating parameters and count their number

print("NDF = ",npar)
print("chi^2/NDF = ",framedM.chiSquare(npar))

chi2ndf = framedM.chiSquare(npar)

# Plot Labels

In [ ]:
# Add text to frame
latex = ROOT.TLatex()
lum = "#splitline{Generic Events}{#scale[0.5]{#int}#it{L} dt=200 fb^{-1}}"
latex.SetNDC()
latex.SetTextSize(0.03)

PerBin = ((xR-xL)/(bin))*1000
y_axis = r'$Entries/(\; {width:.2f}\;MeV/c^2)$'.format(width = PerBin)

framedM.SetTitle("")
framedM.GetYaxis().SetTitle(y_axis)
# framedM.GetYaxis().SetRangeUser(0,22000)
framedM.GetXaxis().SetTitle("")
# framedM.GetXaxis().CenterTitle(ROOT.ktrue)
framedM.Draw()

# The line below prints the luminosity onto the screen.
latex.DrawLatex(0.18,0.89, lum)

# Parameter Box

In [ ]:
xIF1 = pad1.GetUxmin() + pad1.GetLeftMargin()
xIF2 = pad1.GetUxmax() - pad1.GetRightMargin()
yIF1 = pad1.GetUymin() + pad1.GetBottomMargin()
yIF2 = pad1.GetUymax() - pad1.GetTopMargin()
dx = xIF2 - xIF1
dy = yIF2 - yIF1
x1 = xIF1 + 0.4*dx
x2 = xIF2
y1 = yIF1 + 0.75*dy
y2 = 0.93
legend = ROOT.TLegend(x1, y1, x2, y2)

chi2 = r'$\chi^2 /ndf = {chi:.2f}$'.format(chi = chi2ndf) 
Nsig_para = r'{sig:.2f} $\pm$ {error:.2f}'.format(sig = nsig.getValV(), error = nsig.getError())
Nbkg_para = r'{bkg:.2f} $\pm$ {errorb:.2f}'.format(bkg = nbkg.getValV(), errorb = nbkg.getError())

legend.AddEntry(framedM.FindObject("data"),"data","pe")
legend.AddEntry("", chi2, "")
legend.AddEntry("Signal", "D^{0} #rightarrow K^{-} #pi^{+}", "pl" )
legend.AddEntry("", Nsig_para, "")
legend.AddEntry("Background", "Background", "pl" )
legend.AddEntry("", Nbkg_para, "")
legend.AddEntry("model", "Total Fit", "pl" )
legend.AddEntry("", " ", "")

legend.SetBorderSize(0)
legend.SetTextFont(132)
legend.SetTextSize(0.03)
legend.SetNColumns(2)
legend.SetMargin(0.1)
legend.Draw()

# Show Parameter

In [ ]:
# Verbose printing: Basic info, values of constant parameters, initial and
# final values of floating parameters, global correlations
r.Print("v")

# Residual

In [ ]:
pad2.cd()
# Construct a histogram with the residuals of the data w.r.t. the curve
hresid = framedM.residHist()
for iPoint in range(hresid.GetN()):
    hresid.SetPointEYlow(iPoint, 0.0)
    hresid.SetPointEYhigh(iPoint, 0.0)

hresid.SetFillColor(38)

# Create a new frame to draw the residual distribution and add the distribution to the frame
frame1 = dM.frame(RooFit.Title("Residual Distribution"))
frame1.addPlotable(hresid, "B")
frame1.Draw()

# Plot Titles
#---------------
frame1.SetTitle("")

# Y-axis
frame1.SetYTitle("Residual")
# frame1.GetYaxis().CenterTitle(true)
frame1.GetYaxis().SetTitleSize(0.10)
frame1.GetYaxis().SetLabelSize(0.10)

# X-axis
x_axis = r"$\Delta m  [GeV/c^{2}]$"
frame1.SetXTitle(x_axis)
# frame1.GetXaxis().CenterTitle(true)
frame1.GetXaxis().SetTitleSize(0.13)
frame1.GetXaxis().SetLabelSize(0.10)

frame1.SetTitleOffset(0.5, "Y")

# Draw Horizontal Line On Residual Plot
#----------------------------------------
# double xmin1 = xIF1; 
# double xmax1 = 1.0;
# TLine *line11 = new TLine(xIF1,0.0,xIF2,0.0);
# line11->SetLineColor(kRed); line11->SetLineWidth(3);
# line11->Draw("SAME");

c.cd()

In [ ]:
c.Draw()